<a href="https://colab.research.google.com/github/viicctor91/me.github.io/blob/main/Projeto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJETO FINAL

In [ ]:
!pip install pandas_profiling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from tqdm import tqdm
from sklearn.model_selection import train_test_split
#from pandas_profiling import ProfileReport
%matplotlib inline

pd.set_option('display.max_columns', None)

In [ ]:
pd.set_option('display.max_rows', None)
#pd.reset_option('display.max_rows')

In [ ]:
#lendo arquivos
df_sales = pd.read_csv("item_sales.csv")

In [ ]:
df_prices = pd.read_csv("item_prices.csv")

In [ ]:
df_calendar = pd.read_csv("daily_calendar_with_events.csv")

-------

### Dicionário de dados

File 1 – daily_calendar_with_events.csv

| Name | Description |
| --- | --- |
| date | date in y-m-d format |
| weekday |	day of the week |
| weekday_int |	numeric day of the week (Saturday day 1, Friday day 7) |
| d	| day identifier |
| event | if the date includes an event, the name of this event (only a few are included) |

File 2 – item_prices.csv

| Name | Description |
| --- | --- |
| item | product id |
| category | product category |
| store_code | alphanumeric code of the store |
| yearweek | date period for the price (year-week format) |
| sell_price |	price for the product “item” for the period in “yearweek”. * |

* Prices are provided per week (average across 7 days). If not available, there were no sales for the product during that week ludes an event, the name of this event (only a few are included).

File 3 – item_sales.csv

| Name | Description |
| --- | --- |
| id | sales series id (combination of item + store_code) |
| item | product id |
| category | product category |
| department | department id (different identifier for different stores) |
| store | store name |
| store_code | store id |
| region | region |
| d_1,d_2,d_... | number of units sold per day |

------

# Item Prices

## Pré-processamento e exploração de dados

In [ ]:
#Monstra Colonas do datafrane item_prices
cols_df_price = pd.DataFrame(df_prices.columns, columns=['Column Names'])

display(cols_df_price)

In [ ]:
# Primeiras cinco linhas do df
df_prices.head().T

In [ ]:
# Cinco ultimas linhas
df_prices.tail().T

In [ ]:
# Dimensão do df
df_prices.shape

In [ ]:
df_prices.dtypes

In [ ]:
df_prices.describe()

In [ ]:
df_prices.info(verbose=True)

In [ ]:
#verificando valores em falta
soma_missings_por_coluna = df_prices.isnull().sum()
soma_missings_por_coluna

### Tratamento da coluna Yearweek

In [ ]:
# Convertendo a coluna 'yearweek' para o tipo string
df_prices['yearweek'] = df_prices['yearweek'].astype(str)

# Remover linhas com valores vazios na coluna 'yearweek'
df_prices.dropna(subset=['yearweek'], inplace=True)

# Elimine as linhas que contenham a string 'nan' na coluna 'yearweek'
df_prices = df_prices[~df_prices['yearweek'].isin(['nan'])]

# Mantenha apenas os 6 primeiros caracteres na coluna 'yearweek'
df_prices['yearweek'] = df_prices['yearweek'].str.slice(0, 6)

# Ordenando o dataframe pela coluna yearweek
df_prices.sort_values('yearweek', inplace=True)

# Exiba o DataFrame reordenado
print(df_prices)

                        item       category store_code yearweek  sell_price
2353758  HOME_&_GARDEN_2_201  HOME_&_GARDEN      NYC_4   201105      6.8375
58703       ACCESORIES_1_259     ACCESORIES      NYC_1   201105      1.7556
2098440     ACCESORIES_1_144     ACCESORIES      NYC_4   201105     13.3000
301935   HOME_&_GARDEN_2_215  HOME_&_GARDEN      NYC_1   201105      2.3375
571591     SUPERMARKET_3_224    SUPERMARKET      NYC_1   201105      5.1360
...                      ...            ...        ...      ...         ...
1148814    SUPERMARKET_2_119    SUPERMARKET      NYC_2   201617      3.2040
1952627    SUPERMARKET_3_328    SUPERMARKET      NYC_3   201617      4.6560
1664598  HOME_&_GARDEN_2_240  HOME_&_GARDEN      NYC_3   201617      6.2125
2367499  HOME_&_GARDEN_2_254  HOME_&_GARDEN      NYC_4   201617      5.5500
835846      ACCESORIES_2_119     ACCESORIES      NYC_2   201617      3.9501

[2418920 rows x 5 columns]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Divindo a Coluna yearweek em duas (Year, Week)

In [ ]:
# Extrair o ano e a semana
#df_prices['year'] = pd.to_datetime(df_prices['yearweek'].str[:4], format='%Y')
#df_prices['week'] = df_prices['yearweek'].str[-2:].astype(int)


In [ ]:
# Convertendo a coluna 'yearweek' em datetime
#df_prices['yearweek'] = pd.to_datetime(df_prices['yearweek'] + '0', format='%Y%W%w')


In [ ]:
# Verificando se ainda existe valores nulos no df
df_prices.isnull().sum()/len(df_prices)*100

In [ ]:
#verificando valores duplicador no df
df_prices.duplicated().sum()/len(df_prices)*100

### Função para observar cada coluna do df

In [ ]:
def Price_func(data, column, count=True):
    unique_values = data[column].unique()
    null_count = data[column].isnull().sum()
    value_counts = data[column].value_counts()

    print(f'Quantidade de valores únicos: {len(unique_values)}')
    print(f'\nQuais são os valores únicos: {unique_values}')
    print(f'\nQuantidade de valores nulos: {null_count}')
    print(f'\nQuantidade por opção: \n{value_counts}')

#### Coluna Item

In [ ]:
Price_func(df_prices, 'item')

#### Coluna Category

In [ ]:
Price_func(df_prices, 'category')

#### Coluna Store_code

In [ ]:
Price_func(df_prices, 'store_code')

Quantidade de valores únicos: 10

Quais são os valores únicos: ['NYC_1' 'NYC_2' 'NYC_3' 'NYC_4' 'BOS_1' 'BOS_2' 'BOS_3' 'PHI_1' 'PHI_2'
 'PHI_3']

Quantidade de valores nulos: 0

Quantidade por opção: 
BOS_2    713960
BOS_1    712527
NYC_1    711073
PHI_3    708747
NYC_3    706585
BOS_3    703682
NYC_4    691375
PHI_2    690546
PHI_1    678209
NYC_2    649002
Name: store_code, dtype: int64


#### Coluna Yearweek

In [ ]:
Price_func(df_prices, 'yearweek')

#### Coluna Sell_price

In [ ]:
Price_func(df_prices, 'sell_price')

#### Coluna year

In [ ]:
Price_func(df_prices, 'year')

#### Coluna Week

In [ ]:
Price_func(df_prices, 'week')

In [ ]:
# Calcular a contagem de ocorrências de cada valor da coluna 'week'
week_counts = df_prices['week'].value_counts()

# Calcular a porcentagem de ocorrências de cada valor
week_percentages = week_counts / len(df_prices) * 100

# Exibir as porcentagens de ocorrências
print(week_percentages)

##### Removendo semana Zero da coluna week

In [ ]:
# Verificar as linhas que possuem semana igual a 0
week_zero_rows = df_prices[df_prices['week'] == 0]

# Remover as linhas com semana igual a 0
df_prices = df_prices.drop(week_zero_rows.index)

# Resetar o índice do DataFrame
df_prices = df_prices.reset_index(drop=True)

# Exibir o DataFrame resultante
print(df_prices)

In [ ]:
price_stats = df_prices.groupby("category")["sell_price"].agg(["mean", "median", "std"])
print(price_stats)

In [ ]:

# Agrupar os dados pelo ano e semana e calcular as estatísticas descritivas dos preços
price_stats = df_prices.groupby(["year", "week"])["sell_price"].agg(["mean", "median", "std"])
print(price_stats)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_prices["year"], df_prices["sell_price"])
plt.xlabel("Data")
plt.ylabel("Preço")
plt.title("Variação de Preços ao Longo do Tempo")
plt.show()

In [ ]:
df_prices.head()

# Item Sales

## Pré-processamento e exploração de dados

In [ ]:
cols_df_sales = pd.DataFrame(df_sales.columns, columns=['Column Names'])

display(cols_df_sales)

,Column Names
0,id
1,item
2,category
3,department
4,store
...,...
1915,d_1909
1916,d_1910
1917,d_1911
1918,d_1912


In [ ]:
#primeiras 5 linhas do df_sales
df_sales.head()

In [ ]:
# Cinco ultimas linhas
df_sales.tail()

In [ ]:
df_sales.shape

In [ ]:
df_sales.dtypes

In [ ]:
df_sales.describe()

In [ ]:
#df_sales.info(verbose=True)

In [ ]:
#verificando valores em falta
soma_missings_por_coluna = df_sales.isnull().sum()
soma_missings_por_coluna

id            0
item          0
category      0
department    0
store         0
             ..
d_1909        0
d_1910        0
d_1911        0
d_1912        0
d_1913        0
Length: 1920, dtype: int64

### Separando o item Sales por dados Categoricos

In [ ]:
df_sales_cat = df_sales.iloc[:,0:7]
df_sales_cat

,id,item,category,department,store,store_code,region
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York
...,...,...,...,...,...,...,...
1319,HOME_&_GARDEN_2_224_NYC_1,HOME_&_GARDEN_2_224,HOME_&_GARDEN,HOME_&_GARDEN_2,Greenwich_Village,NYC_1,New York
1320,HOME_&_GARDEN_2_225_NYC_1,HOME_&_GARDEN_2_225,HOME_&_GARDEN,HOME_&_GARDEN_2,Greenwich_Village,NYC_1,New York
1321,HOME_&_GARDEN_2_226_NYC_1,HOME_&_GARDEN_2_226,HOME_&_GARDEN,HOME_&_GARDEN_2,Greenwich_Village,NYC_1,New York
1322,HOME_&_GARDEN_2_227_NYC_1,HOME_&_GARDEN_2_227,HOME_&_GARDEN,HOME_&_GARDEN_2,Greenwich_Village,NYC_1,New York


### Separando o item Sales por dados Numericos

In [ ]:
df_units_pday = df_sales.iloc[:, 7:].T.reset_index().rename(columns={"index": "d"})
df_units_pday

d    0    1    2    3    4    5    6     7    8    9   10   11  \
0        d_1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  12.0  2.0  0.0  0.0  0.0   
1        d_2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  15.0  0.0  0.0  0.0  2.0   
2        d_3  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  7.0  1.0  0.0  0.0   
3        d_4  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  3.0  0.0  0.0  0.0   
4        d_5  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0   
...      ...  ...  ...  ...  ...  ...  ...  ...   ...  ...  ...  ...  ...   
1908  d_1909  1.0  1.0  1.0  0.0  1.0  0.0  1.0   4.0  0.0  0.0  0.0  0.0   
1909  d_1910  3.0  0.0  0.0  1.0  2.0  0.0  0.0   6.0  0.0  0.0  0.0  1.0   
1910  d_1911  0.0  0.0  1.0  3.0  2.0  2.0  0.0   3.0  0.0  2.0  0.0  0.0   
1911  d_1912  1.0  0.0  1.0  7.0  2.0  0.0  1.0   2.0  0.0  0.0  0.0  0.0   
1912  d_1913  1.0  0.0  1.0  2.0  4.0  0.0  1.0   1.0  0.0  2.0  0.0  0.0   

       12   13   14    15   16   17    18   19   20   21   22   23   24   25  \
0     0.0  0.0  4.0   5.0  0.0  0.0   0.0  0.0  0.0  2.0  2.0  0.0  0.0  0.0   
1     0.0  0.0  0.0   1.0  0.0  0.0   0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0   
2     0.0  0.0  0.0   3.0  0.0  0.0   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
3     0.0  0.0  5.0   0.0  0.0  0.0   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
4     0.0  0.0  0.0  15.0  0.0  0.0   0.0  0.0  0.0  0.0  2.0  0.0  0.0  0.0   
...   ...  ...  ...   ...  ...  ...   ...  ...  ...  ...  ...  ...  ...  ...   
1908  0.0  0.0  0.0   4.0  0.0  0.0   0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0   
1909  0.0  0.0  0.0   0.0  0.0  0.0  11.0  0.0  0.0  1.0  1.0  0.0  2.0  0.0   
1910  0.0  3.0  2.0  15.0  2.0  0.0   4.0  0.0  0.0  0.0  0.0  2.0  2.0  0.0   
1911  1.0  1.0  5.0   1.0  1.0  0.0   6.0  0.0  1.0  0.0  4.0  0.0  2.0  1.0   
1912  1.0  1.0  4.0   6.0  6.0  0.0  12.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0   

       26   27   28    29   30    31   32   33   34   35   36   37   38   39  \
0     0.0  0.0  2.0   0.0  0.0   9.0  0.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0   
1     0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0   
2     0.0  0.0  3.0   0.0  0.0   5.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
3     0.0  0.0  0.0   0.0  0.0  16.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
4     0.0  1.0  0.0   0.0  0.0   8.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...   ...  ...  ...   ...  ...   ...  ...  ...  ...  ...  ...  ...  ...  ...   
1908  0.0  1.0  1.0   7.0  0.0   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
1909  0.0  1.0  4.0   7.0  0.0   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
1910  0.0  1.0  0.0  19.0  0.0   2.0  0.0  0.0  0.0  0.0  3.0  1.0  0.0  0.0   
1911  2.0  1.0  0.0   3.0  0.0   4.0  0.0  0.0  0.0  0.0  1.0  4.0  0.0  0.0   
1912  0.0  1.0  3.0  11.0  0.0   4.0  0.0  0.0  0.0  2.0  0.0  2.0  0.0  0.0   

       40   41   42   43   44   45   46    47   48   49   50   51   52   53  \
0     0.0  0.0  0.0  3.0  0.0  0.0  1.0   0.0  0.0  2.0  0.0  0.0  0.0  0.0   
1     0.0  0.0  0.0  3.0  0.0  0.0  2.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2     0.0  0.0  0.0  2.0  0.0  0.0  3.0   0.0  0.0  1.0  0.0  0.0  0.0  0.0   
3     0.0  0.0  0.0  1.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4     0.0  0.0  0.0  1.0  0.0  0.0  1.0   0.0  0.0  1.0  0.0  0.0  0.0  0.0   
...   ...  ...  ...  ...  ...  ...  ...   ...  ...  ...  ...  ...  ...  ...   
1908  0.0  1.0  0.0  0.0  0.0  0.0  0.0  19.0  1.0  1.0  0.0  0.0  0.0  0.0   
1909  0.0  0.0  0.0  0.0  0.0  0.0  0.0  10.0  0.0  1.0  1.0  1.0  0.0  0.0   
1910  2.0  0.0  0.0  1.0  1.0  1.0  0.0  20.0  1.0  0.0  0.0  0.0  0.0  0.0   
1911  0.0  0.0  0.0  0.0  0.0  1.0  0.0  10.0  0.0  0.0  0.0  0.0  0.0  0.0   
1912  0.0  0.0  0.0  6.0  0.0  1.0  0.0  21.0  0.0  1.0  0.0  0.0  0.0  1.0   

       54   55   56   57   58   59   60   61   62   63   64   65   66   67  \
0     0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
1     0.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

### Função para observar cada coluna do df

In [ ]:
def sales_func(data, column, count=True):
    unique_values = data[column].unique()
    null_count = data[column].isnull().sum()
    value_counts = data[column].value_counts()

    print(f'Quantidade de valores únicos: {len(unique_values)}')
    print(f'\nQuais são os valores únicos: {unique_values}')
    print(f'\nQuantidade de valores nulos: {null_count}')
    print(f'\nQuantidade por opção: \n{value_counts}')

#### Coluna id

In [ ]:
sales_func(df_sales_cat, 'id')

#### Coluna item

In [ ]:
sales_func(df_sales_cat, 'item')

In [ ]:
sales_func(df_sales_cat, 'category')

In [ ]:
sales_func(df_sales_cat, 'department')

In [ ]:
sales_func(df_sales_cat, 'store')

In [ ]:
sales_func(df_sales_cat, 'store_code')

In [ ]:
sales_func(df_sales_cat, 'region')

In [ ]:
df_sales_cat.head(10)

In [ ]:
# Agrupar por 'region' e listar todas as lojas associadas a cada região
lojas_por_regiao = df_sales.groupby('region')['store'].unique()

print(lojas_por_regiao)



In [ ]:
df_group = df_sales.groupby("region")["store_code"].value_counts()
print(df_group)

In [ ]:
df_sales['department'].unique()

In [ ]:
df_sales['id'].unique()

# Juntando as Tabelas

Verifique a junção da tabela df_sales_melted com df_calendar. Certifique-se de que ambas as tabelas têm a mesma quantidade de linhas únicas para o campo d.

In [ ]:
unique_sales_d = df_sales_melted['d'].nunique()
unique_calendar_d = df_calendar['d'].nunique()

print(f"Unique d in df_sales_melted: {unique_sales_d}")
print(f"Unique d in df_calendar: {unique_calendar_d}")

Unique d in df_sales_melted: 1913
Unique d in df_calendar: 1913


Verifique os valores únicos de store_code na tabela df_prices e df_sales_melted:

In [ ]:
unique_prices_store_code = df_prices['store_code'].unique()
unique_sales_store_code = df_sales_melted['store_code'].unique()

print(f"Unique store_code in df_prices: {unique_prices_store_code}")
print(f"Unique store_code in df_sales_melted: {unique_sales_store_code}")


Unique store_code in df_prices: ['NYC_1' 'NYC_2' 'NYC_3' 'NYC_4' 'BOS_1' 'BOS_2' 'BOS_3' 'PHI_1' 'PHI_2'
 'PHI_3']
Unique store_code in df_sales_melted: ['NYC_1' 'NYC_2' 'NYC_3' 'NYC_4' 'BOS_1' 'BOS_2' 'BOS_3' 'PHI_1' 'PHI_2'
 'PHI_3']



## Transformação de df_sales:

In [ ]:
print(df_sales['store_code'].unique())


In [ ]:
df_sales = df_sales.reset_index(drop=True)


In [ ]:
print(df_sales['store_code'].dtype)
print(df_sales['store_code'].unique())


In [ ]:
print(df_sales[df_sales['store_code'] == 'NYC_2'].head())


In [ ]:
# 1. Defina o tamanho do chunk
chunk_size = 500

# 2. Divida as colunas d_ em chunks
d_cols = [f'd_{i}' for i in range(1, 1914)]
d_chunks = [d_cols[i:i + chunk_size] for i in range(0, len(d_cols), chunk_size)]

# 3. Realize a operação melt em cada chunk
melted_dfs = []
for d_chunk in d_chunks:
    melted_df = df_sales.melt(id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'],
                              value_vars=d_chunk,
                              var_name='d',
                              value_name='sold_units')
    melted_dfs.append(melted_df)

# 4. Concatene os resultados
df_sales_melted = pd.concat(melted_dfs, axis=0)

# Verifique os resultados
print(df_sales_melted.shape)
print(df_sales_melted['store_code'].value_counts())


(58327370, 9)
NYC_1    5832737
NYC_2    5832737
NYC_3    5832737
NYC_4    5832737
BOS_1    5832737
BOS_2    5832737
BOS_3    5832737
PHI_1    5832737
PHI_2    5832737
PHI_3    5832737
Name: store_code, dtype: int64


## Juntar com daily_calendar_with_events:

In [ ]:
# Converter a coluna 'd' em ambos os dataframes para int64
df_calendar['d'] = df_calendar['d'].str.extract('(\d+)').astype(int)

# Suponhamos que cada chunk terá 50000 linhas
chunk_size = 50000
chunks = [x for x in range(0, df_sales_melted.shape[0], chunk_size)]

# Lista para armazenar os resultados de cada chunk processado
dfs = []

for start in chunks:
    # Pega o pedaço do DataFrame
    subset = df_sales_melted.iloc[start:start+chunk_size].copy()

    # Converter a coluna 'd' em inteiro para subset
    subset['d'] = subset['d'].str.extract('(\d+)').astype(int)

    # Junção com df_calendar
    merged_subset = pd.merge(subset, df_calendar, on='d', how='left')

    # Adicionar o resultado na lista
    dfs.append(merged_subset)

# Concatenar todos os resultados
df_sales_calendar = pd.concat(dfs, ignore_index=True)


## Extrair o 'yearweek' a partir da coluna 'date':

In [ ]:
df_sales_calendar['date'] = pd.to_datetime(df_sales_calendar['date'])
df_sales_calendar['yearweek'] = df_sales_calendar['date'].dt.strftime('%Y%U').astype(int)


## Juntar com item_prices:

In [ ]:
df_prices['yearweek'] = df_prices['yearweek'].astype(int)

IntCastingNaNError: ignored

In [ ]:
def merge_in_chunks(df_main, df_secondary, on_columns, n_chunks=20, how='left'):
    chunk_size = len(df_main) // n_chunks
    merged_dfs = []

    # Envolva o intervalo em tqdm() para obter a barra de progresso
    for i in tqdm(range(n_chunks)):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if i != n_chunks - 1 else None  # Para pegar tudo até o final no último segmento

        chunk = df_main.iloc[start_idx:end_idx]
        merged_chunk = pd.merge(chunk, df_secondary, on=on_columns, how=how)

        merged_dfs.append(merged_chunk)

    return pd.concat(merged_dfs)

on_columns = ['item', 'store_code', 'yearweek']
df_final = merge_in_chunks(df_sales_calendar, df_prices, on_columns, n_chunks=20)



100%|██████████| 20/20 [01:51<00:00,  5.59s/it]


In [ ]:
df_final.head()

,id,item,category_x,department,store,store_code,region,d,sales,date,weekday,weekday_int,event,yearweek,category_y,sell_price
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,1,0.0,2011-01-29,Saturday,1,NaN,201104,NaN,NaN
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,1,0.0,2011-01-29,Saturday,1,NaN,201104,NaN,NaN
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,1,0.0,2011-01-29,Saturday,1,NaN,201104,NaN,NaN
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,1,0.0,2011-01-29,Saturday,1,NaN,201104,NaN,NaN
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,1,0.0,2011-01-29,Saturday,1,NaN,201104,NaN,NaN


# Verificar quais  produtos que não são mais tão populares

In [ ]:
# Transformando a coluna 'date' em datetime
df_final['date'] = pd.to_datetime(df_final['date'])

# Definindo o período de tempo para os últimos três meses e três meses antes disso
end_date = df_final['date'].max()
start_date_last_3_months = end_date - pd.DateOffset(months=3)
start_date_prev_3_months = start_date_last_3_months - pd.DateOffset(months=3)

# Filtrando o DataFrame para os produtos vendidos nos últimos 6 meses
last_6_months_sales = df_final[df_final['date'] > start_date_prev_3_months]

# Calculando a média de vendas por produto nos últimos três meses e nos três meses anteriores
avg_sales_last_3_months = last_6_months_sales[last_6_months_sales['date'] > start_date_last_3_months].groupby('item')['sold_units'].mean()
avg_sales_prev_3_months = last_6_months_sales[last_6_months_sales['date'] <= start_date_last_3_months].groupby('item')['sold_units'].mean()

# Juntando as duas séries e calculando a diferença percentual
sales_comparison = pd.concat([avg_sales_prev_3_months, avg_sales_last_3_months], axis=1, keys=['Prev_3_months', 'Last_3_months'])
sales_comparison['Sales_decline_percent'] = ((sales_comparison['Prev_3_months'] - sales_comparison['Last_3_months']) / sales_comparison['Prev_3_months']) * 100

# Filtrando produtos que tiveram uma queda de, digamos, mais de 20% nas vendas
less_popular_products = sales_comparison[sales_comparison['Sales_decline_percent'] > 20]

# Ordenando o DataFrame pelo declínio percentual das vendas de forma descendente
less_popular_products_sorted = less_popular_products.sort_values(by='Sales_decline_percent', ascending=False)

print(less_popular_products_sorted)


                     Prev_3_months  Last_3_months  Sales_decline_percent
item                                                                    
SUPERMARKET_3_441         0.114130       0.000000             100.000000
SUPERMARKET_3_444         0.208696       0.000000             100.000000
HOME_&_GARDEN_2_108       0.221739       0.001099              99.504417
ACCESORIES_1_236          0.634783       0.008791              98.615084
HOME_&_GARDEN_2_202       0.079348       0.001099              98.615084
...                            ...            ...                    ...
ACCESORIES_1_002          0.331522       0.263736              20.446766
HOME_&_GARDEN_1_194       1.377174       1.096703              20.365664
SUPERMARKET_3_505         3.017391       2.407692              20.206163
HOME_&_GARDEN_2_087       0.067391       0.053846              20.099256
HOME_&_GARDEN_1_005       1.525000       1.219780              20.014412

[290 rows x 3 columns]


## Verificar os produtos mais populares entre as cidades

In [ ]:
N = 5

# Agrupando por loja, produto e somando as vendas
store_product_sales = df_final.groupby(['store', 'item'])['sold_units'].sum().reset_index()

# Lista para armazenar os produtos mais populares por loja
top_products_list = []

for store in store_product_sales['store'].unique():
    store_sales = store_product_sales[store_product_sales['store'] == store]

    # Ordenando os produtos por vendas e pegando os top N produtos mais populares
    top_products = store_sales.sort_values(by='sold_units', ascending=False).head(N)

    top_products_list.append(top_products)

# Concatenando os resultados para criar o DataFrame final
top_products_per_store = pd.concat(top_products_list, axis=0)

print(top_products_per_store[['store', 'item', 'sold_units']])




                   store               item  sold_units
2810            Back_Bay  SUPERMARKET_3_586      150122
2314            Back_Bay  SUPERMARKET_3_090      114854
2476            Back_Bay  SUPERMARKET_3_252       86632
2779            Back_Bay  SUPERMARKET_3_555       77673
2601            Back_Bay  SUPERMARKET_3_377       73797
5363            Brooklyn  SUPERMARKET_3_090       52264
5859            Brooklyn  SUPERMARKET_3_586       32557
5860            Brooklyn  SUPERMARKET_3_587       24333
5525            Brooklyn  SUPERMARKET_3_252       23417
6078            Brooklyn  SUPERMARKET_3_808       21282
8412   Greenwich_Village  SUPERMARKET_3_090      127203
8908   Greenwich_Village  SUPERMARKET_3_586       87691
8574   Greenwich_Village  SUPERMARKET_3_252       74971
8442   Greenwich_Village  SUPERMARKET_3_120       61899
9035   Greenwich_Village  SUPERMARKET_3_714       54080
11957             Harlem  SUPERMARKET_3_586       63416
11623             Harlem  SUPERMARKET_3_252     

In [27]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58327370 entries, 0 to 2916377
Data columns (total 16 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           object        
 1   item         object        
 2   category_x   object        
 3   department   object        
 4   store        object        
 5   store_code   object        
 6   region       object        
 7   d            int64         
 8   sold_units   int64         
 9   date         datetime64[ns]
 10  weekday      object        
 11  weekday_int  int64         
 12  event        object        
 13  yearweek     int64         
 14  category_y   object        
 15  sell_price   float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(10)
memory usage: 7.4+ GB


In [28]:
df_final.to_csv("df_final.csv", index=False)


## Modificando a Coluna ID do Dataframe df_sales

In [ ]:
# modificando o nome do item Home & Gardem para poder dividir a coluna e extrair o id
df_sales['id'] = df_sales['id'].str.replace('HOME_&_GARDEN_', 'HOME&GARDEN_')

In [ ]:
# Extrair os valores entre a segunda e a terceira ocorrência do caractere de sublinhado
df_sales['id'] = df_sales['id'].str.split('_').str[2]


In [ ]:
df_sales.to_csv('arquivos/df_sales.csv', index=False)

In [ ]:
sales_func(df_sales, 'item')

In [ ]:

# Calcular a soma das vendas diárias para cada produto (linha) e armazenar em uma nova coluna "total_units_sold"
df_sales["total_units_sold"] = df_sales.iloc[:, 7:].sum(axis=1)

# Ordenar o dataframe em ordem decrescente com base na coluna "total_units_sold"
df_sorted = df_sales.sort_values(by="total_units_sold", ascending=False)

# Exibir os produtos com as maiores quantidades de unidades vendidas
print("Produtos mais vendidos por unidades:")
print(df_sorted[["item", "total_units_sold"]].head(10))  # Exibe os 10 primeiros produtos

In [ ]:
# Filtrar os produtos que começam com o nome "accessories"
df_accessories = df_sales[df_sales["item"].str.startswith("ACCESORIES")]

# Calcular a soma das vendas diárias para cada produto (linha) e armazenar em uma nova coluna "total_units_sold"
df_accessories["total_units_sold"] = df_accessories.iloc[:, 7:].sum(axis=1)

# Ordenar o dataframe em ordem decrescente com base na coluna "total_units_sold"
df_sorted = df_accessories.sort_values(by="total_units_sold", ascending=False)

# Exibir os 10 produtos mais vendidos
print("Top 10 produtos mais vendidos com nome começando por 'accessories':")
print(df_sorted[["item", "total_units_sold"]].head(10))

In [ ]:
# Filtrar os produtos que começam com o nome "accessories"
df_accessories = df_sales[df_sales["item"].str.startswith("ACCESORIES")]

# Calcular a soma das vendas diárias para cada produto (linha) e armazenar em uma nova coluna "total_units_sold"
df_accessories["total_units_sold"] = df_accessories.iloc[:, 7:].sum(axis=1)

# Agrupar os dados por item e loja, calculando a soma das unidades vendidas em cada combinação
df_grouped = df_accessories.groupby(["item", "store"])["total_units_sold"].sum().reset_index()

# Encontrar a loja com a maior quantidade de vendas para cada item
idx = df_grouped.groupby("item")["total_units_sold"].idxmax()
df_top_stores = df_grouped.loc[idx]

# Mesclar o DataFrame df_top_stores com o DataFrame df_sales para adicionar a coluna "store_code"
df_top_stores = df_top_stores.merge(df_sales[["item", "store", "store_code"]], on=["item", "store"], how="left")

# Ordenar os resultados pelo total de unidades vendidas em ordem decrescente
df_top_stores = df_top_stores.sort_values("total_units_sold", ascending=False)

# Exibir os 10 produtos mais vendidos com a respectiva loja e store_code que vendeu mais
print("Top 10 produtos mais vendidos com nome começando por 'accessories', ordenados pelo total de unidades vendidas:")
print(df_top_stores.head(10))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Selecionar as colunas de dias de venda
df_sales_days = df_sales.iloc[:, 6:]

# Calcular a média das vendas diárias para cada produto
df_sales_mean = df_sales_days.mean(axis=1)

# Adicionar a coluna de média de vendas ao DataFrame original
df_sales["mean_sales"] = df_sales_mean

# Criar um DataFrame para armazenar os coeficientes de regressão linear
df_coefficients = pd.DataFrame(columns=["Produto", "Coeficiente"])

# Iterar sobre os produtos
for item in df_sales["item"].unique():
    # Filtrar o DataFrame para o produto atual
    df_item = df_sales[df_sales["item"] == item]

    # Verificar se há dados suficientes para a regressão linear
    if len(df_item) > 1:
        # Extrair as vendas e os dias
        X = df_item.index.values.reshape(-1, 1)
        y = df_item["mean_sales"].values

        # Criar o modelo de regressão linear
        model = LinearRegression()

        # Ajustar o modelo aos dados de vendas
        model.fit(X, y)

        # Obter o coeficiente da regressão linear
        coef = model.coef_[0]

        # Armazenar o coeficiente no DataFrame
        df_coefficients = df_coefficients.append({"Produto": item, "Coeficiente": coef}, ignore_index=True)

# Ordenar o DataFrame pelos coeficientes em ordem decrescente
df_declining_sorted = df_coefficients.sort_values("Coeficiente", ascending=True)

# Criar o gráfico de barras
plt.figure(figsize=(12, 6))
plt.barh(df_declining_sorted["Produto"], df_declining_sorted["Coeficiente"])
plt.xlabel("Coeficiente de Regressão Linear")
plt.ylabel("Produto")
plt.title("Produtos com Maiores Quedas nas Vendas")
plt.tight_layout()

# Exibir o gráfico
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Carregar o arquivo item_sales.csv em um DataFrame
df_sales = pd.read_csv("item_sales.csv")

# Selecionar as colunas de dias de venda
df_sales_days = df_sales.iloc[:, 7:]  # Selecionar as colunas de dias de venda a partir da coluna 7

# Calcular a variação percentual das vendas para cada produto
df_sales_percent_change = df_sales_days.pct_change(axis=1).mean(axis=1) * 100

# Adicionar a coluna de variação percentual ao DataFrame original
df_sales["percent_change"] = df_sales_percent_change

# Filtrar os produtos com maiores quedas nas vendas
df_declining_products = df_sales[df_sales["percent_change"] < 0]

# Ordenar os produtos por variação percentual em ordem crescente
df_declining_products = df_declining_products.sort_values("percent_change", ascending=True)

# Selecionar os 10 produtos com maiores quedas nas vendas
top_declining_products = df_declining_products.head(10)

# Criar o gráfico de barras
plt.figure(figsize=(12, 6))
plt.barh(top_declining_products["item"], top_declining_products["percent_change"])
plt.xlabel("Variação Percentual das Vendas")
plt.ylabel("Produto")
plt.title("Produtos com Maiores Quedas nas Vendas")
plt.tight_layout()

# Exibir o gráfico
plt.show()


In [ ]:
import pandas as pd

# Carregar o arquivo
df_sales = pd.read_csv("item_sales.csv")

# Transformar colunas d_1, d_2, ... em linhas
df_sales_melted = df_sales.melt(id_vars=["id", "item", "category", "department", "store", "store_code", "region"],
                                var_name="date",
                                value_name="sales")

# Agora, considerando um período recente, por exemplo, os últimos 30 dias
latest_date = pd.to_datetime(df_sales_melted["date"].str.split("_", expand=True)[1].astype(int).max())
start_date = latest_date - pd.Timedelta(days=30)

# Filtrar as vendas dos últimos 30 dias
recent_sales = df_sales_melted[(df_sales_melted["date"] >= f"d_{start_date.day}") & (df_sales_melted["date"] <= f"d_{latest_date.day}")]

# Calcular a soma de vendas para cada item no período recente
recent_sales_sum = recent_sales.groupby("item")["sales"].sum()

# Identificar produtos menos populares
unpopular_products_recent = recent_sales_sum.sort_values().head(10)

print(unpopular_products_recent)


# Daily Calendar with Events

In [ ]:
df_calendar = pd.read_csv("daily_calendar_with_events.csv")

In [ ]:
df_calendar.shape

In [ ]:
df_calendar.describe()

In [ ]:
cols_df_calendar = pd.DataFrame(df_calendar.columns, columns=['Column Names'])

display(cols_df_calendar)

In [ ]:
df_calendar.head()

In [ ]:
df_calendar.isnull().sum()

In [ ]:
sales_func( df_sales,